In [ ]:
from omegaconf import OmegaConf
from datasets import load_data
import numpy as np

import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from architectures import load_architecture

config = OmegaConf.load("./configs/default_config_linearprobe50.yaml")

backbones=(
    
  'CLIP-convnext_base_w-laion_aesthetic-s13B-b82K',
  'CLIP-convnext_base_w-laion2B-s13B-b82K',
    
  'robust_resnet50',
  'robust_deit_small_patch16_224',
  'robust_vit_base_patch16_224',
  'robust_convnext_base',
  'robust_convnext_tiny',

 'deit_small_patch16_224.fb_in1k', 'robust_resnet50', 
 'vit_base_patch16_224.mae', 'vit_small_patch16_224.augreg_in21k', 
 'convnext_base.fb_in1k', 'resnet50.a1_in1k', 'robust_vit_base_patch16_224', 
 'vit_base_patch16_224.sam_in1k', 'vit_small_patch16_224.dino', 
 'convnext_base.fb_in22k', 'robust_convnext_base', 'vit_base_patch16_224.augreg_in1k',
 'vit_base_patch16_224.augreg_in21k', 'vit_base_patch16_224.dino', 
 'vit_base_patch16_clip_224.laion2b', 'convnext_tiny.fb_in1k', 'robust_convnext_tiny', 
 'robust_deit_small_patch16_224', 'vit_base_patch16_224.dino', 
 'vit_small_patch16_224.augreg_in1k', 'convnext_tiny.fb_in22k'  
   )  

N = 10 # change the classification head to N classes

for backbone in backbones:
    
    config.backbone = backbone
    hp_opt = False
    model = load_architecture(hp_opt, config, N, )
    print('architecture loaded')
    print(model)
    print('###################')
    print('###################')
    print('###################')

Loading OpenCLIP model: convnext_base_w
architecture loaded
CLIPConvNeXtClassifier(
  (backbone): TimmModel(
    (trunk): ConvNeXt(
      (stem): Sequential(
        (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
        (1): LayerNorm2d((128,), eps=1e-06, elementwise_affine=True)
      )
      (stages): Sequential(
        (0): ConvNeXtStage(
          (downsample): Identity()
          (blocks): Sequential(
            (0): ConvNeXtBlock(
              (conv_dw): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
              (norm): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
              (mlp): Mlp(
                (fc1): Linear(in_features=128, out_features=512, bias=True)
                (act): GELU()
                (drop1): Dropout(p=0.0, inplace=False)
                (norm): Identity()
                (fc2): Linear(in_features=512, out_features=128, bias=True)
                (drop2): Dropout(p=0.0, inplace=False)
             

In [2]:
import open_clip

model, _, _ = open_clip.create_model_and_transforms(
    'convnext_base_w',  # architecture
    pretrained=None     # because you’re loading your own checkpoint
)

# Then load the checkpoint you saved
# state_dict = torch.load('/path/to/your/hf_checkpoint.pt', map_location='cpu')
# model.load_state_dict(state_dict, strict=False)


In [3]:
model

CLIP(
  (visual): TimmModel(
    (trunk): ConvNeXt(
      (stem): Sequential(
        (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
        (1): LayerNorm2d((128,), eps=1e-06, elementwise_affine=True)
      )
      (stages): Sequential(
        (0): ConvNeXtStage(
          (downsample): Identity()
          (blocks): Sequential(
            (0): ConvNeXtBlock(
              (conv_dw): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
              (norm): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
              (mlp): Mlp(
                (fc1): Linear(in_features=128, out_features=512, bias=True)
                (act): GELU()
                (drop1): Dropout(p=0.0, inplace=False)
                (norm): Identity()
                (fc2): Linear(in_features=512, out_features=128, bias=True)
                (drop2): Dropout(p=0.0, inplace=False)
              )
              (shortcut): Identity()
              (drop_path): Identity()
  

In [1]:
import timm

vit_models = timm.list_models('*vit_base_patch16_clip_224*', pretrained=True)
for model_name in vit_models:
    print(model_name)

/home/mheuillet/Desktop/robust_training/.venv2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


vit_base_patch16_clip_224.datacompxl
vit_base_patch16_clip_224.dfn2b
vit_base_patch16_clip_224.laion2b
vit_base_patch16_clip_224.laion2b_ft_in1k
vit_base_patch16_clip_224.laion2b_ft_in12k
vit_base_patch16_clip_224.laion2b_ft_in12k_in1k
vit_base_patch16_clip_224.laion400m_e32
vit_base_patch16_clip_224.metaclip_2pt5b
vit_base_patch16_clip_224.metaclip_400m
vit_base_patch16_clip_224.openai
vit_base_patch16_clip_224.openai_ft_in1k
vit_base_patch16_clip_224.openai_ft_in12k
vit_base_patch16_clip_224.openai_ft_in12k_in1k


In [42]:
import torch
state_dict = torch.load('./state_dicts/resnet50_linf_eps4.0.ckpt', weights_only=False)
# 2. Create a new dict with comprehension that excludes certain keys

# Keys to remove exactly
exact_keys_to_remove = {"module.normalizer.new_std", "module.normalizer.new_mean"}

# Prefixes we want to remove if a key starts with them
prefixes_to_remove = ("module.attacker",)  # tuple of prefixes, you can add more if needed

filtered_state_dict = {
    k: v
    for k, v in state_dict["model"].items()
    # Keep this entry only if:
    # 1) it’s not in exact_keys_to_remove, AND
    # 2) it doesn't start with any of the given prefixes
    if k not in exact_keys_to_remove
    and not any(k.startswith(prefix) for prefix in prefixes_to_remove)
}

ckpt = {k.replace('module.model.', ''): v for k, v in filtered_state_dict.items()}

# model = timm.models.resnest50(pretrained=False)
# model.load_state_dict(ckpt)
import timm

model = timm.create_model("resnet50", pretrained=False)
model.load_state_dict(ckpt)

torch.save(model.state_dict(), "./state_dicts/resnet50_linf_eps4.0.pt")


state_dict = torch.load("./state_dicts/resnet50_linf_eps4.0.pt", weights_only=False)
model = timm.create_model("resnet50", pretrained=False)
model.load_state_dict(state_dict)